In [1]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [2]:
import pandas as pd
import us

In [3]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [4]:
from df2gspread import df2gspread as d2g

### datawrapper credentials

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

### setting up google sheets

In [6]:
#scope = ['https://spreadsheets.google.com/feeds']
#credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
#gc = gspread.authorize(credentials)

### get a specific spreadsheet

In [7]:
#spreadsheet_key = "1LXH-tda6l5Mx53bJHUDyDJrAZxifp_0O7F3P47BgtIo"
#book = gc.open_by_key(spreadsheet_key)

In [8]:
#Opening the worksheet by using Worksheet ID
#workbook = gc.open_by_key(spreadsheet_key)
#Selecting which sheet to pulling the data
#sheet = workbook.worksheet('Sheet1')
#Pulling the data and transform it to the data frame
#values = sheet.get_all_values()
#pacifictemps = pd.DataFrame(values[1:], columns = values[0])

In [9]:
import requests
import bs4

In [10]:
state_fips = pd.read_csv("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv")

In [11]:
urls=[]

In [12]:
state_fips['st_lower']=state_fips['stname'].str.lower().str.replace(" ","-")

In [13]:
state_fips['st_lower'][8]='dc'

/var/folders/t2/zm_cy93562b5d1qyf6j1yc1m0000gn/T/ipykernel_56134/2038475284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_fips['st_lower'][8]='dc'


In [14]:
states=state_fips['st_lower']
states_proper=state_fips['stname']

In [15]:
#for state in states:
 #   urls.append("https://www.cdc.gov/childrensmentalhealth/stateprofiles-providers/"+str(state)+"/index.html")

In [16]:
stdata=[]
for wiki in wiki_links:
    wikipages=requests.get(str(wiki))
    imgsoup=bs4.BeautifulSoup(wikipages.text,'html.parser')
    
    
    table=imgsoup.find('table')
    rows=table.find_all('tr')
    for row in rows[1:]:
        cells=row.find_all('td')
        data_dict={
        'county': cells[0].text.strip(),
        'pediatricians': cells[1].text.strip(),
        'psychiatrists': cells[2].text.strip(),
        'fm_physicians': cells[3].text.strip(),
        'licensed_sw': cells[4].text.strip(),
        'psychologists': cells[5].text.strip(),
        'st': str(state)}
        stdata.append(data_dict)
# adding the link is a workaround--if for each state in states_proper I could add the state name, that would be preferable

In [17]:
county_providers=pd.DataFrame(stdata)

In [18]:
county_providers

,county,pediatricians,psychiatrists,fm_physicians,licensed_sw,psychologists,st
0,Autauga,2.2,0.7,7.9,0.7,0.0,alabama
1,Baldwin,5.6,1.6,18.3,5.6,2.7,alabama
2,Barbour,1.8,0.0,12.3,0.0,1.8,alabama
3,Bibb,0.0,0.0,16.9,0.0,0.0,alabama
4,Blount,0.7,0.0,7.3,0.0,0.0,alabama
...,...,...,...,...,...,...,...
3115,Sweetwater,1.6,0.0,7.4,27.9,4.1,wyoming
3116,Teton,16.0,6.9,18.3,48.0,22.9,wyoming
3117,Uinta,3.2,4.8,8.0,30.5,35.3,wyoming
3118,Washakie,0.0,0.0,15.1,25.1,15.1,wyoming


In [19]:
county_providers['st']=county_providers['st'].str.replace("-", " ").str.title()

In [20]:
county_fips=pd.read_csv('data/raw/countyfipstool20190120.csv',dtype=str)

In [21]:
providers_wfips=county_providers.merge(county_fips, left_on=['st','county'],right_on=['sname','cname'],how='left')

In [22]:
providers_clean=providers_wfips.drop(columns=['sname','sid','sfips','saint','cfips'])

In [23]:
providers_clean

,county,pediatricians,psychiatrists,fm_physicians,licensed_sw,psychologists,st,sab,cname,fips
0,Autauga,2.2,0.7,7.9,0.7,0.0,Alabama,AL,Autauga,1001
1,Baldwin,5.6,1.6,18.3,5.6,2.7,Alabama,AL,Baldwin,1003
2,Barbour,1.8,0.0,12.3,0.0,1.8,Alabama,AL,Barbour,1005
3,Bibb,0.0,0.0,16.9,0.0,0.0,Alabama,AL,Bibb,1007
4,Blount,0.7,0.0,7.3,0.0,0.0,Alabama,AL,Blount,1009
...,...,...,...,...,...,...,...,...,...,...
3115,Sweetwater,1.6,0.0,7.4,27.9,4.1,Wyoming,WY,Sweetwater,56037
3116,Teton,16.0,6.9,18.3,48.0,22.9,Wyoming,WY,Teton,56039
3117,Uinta,3.2,4.8,8.0,30.5,35.3,Wyoming,WY,Uinta,56041
3118,Washakie,0.0,0.0,15.1,25.1,15.1,Wyoming,WY,Washakie,56043


In [24]:
shortfips=providers_clean[providers_clean['fips'].str.len()<5].copy()

In [25]:
longfips=providers_clean[providers_clean['fips'].str.len()>4].copy()

In [26]:
shortfips['fips']='0'+shortfips['fips']

In [27]:
providers_counties_fips=pd.concat([shortfips, longfips])

In [28]:
missing_counties=providers_clean[providers_clean['fips'].isnull()].copy()

In [29]:
missing_counties['county']=missing_counties['county'].str.replace('Saint','St.')

In [30]:
missing_counties['county'][350]='Miami-Dade'
missing_counties['st'][307]='District of Columbia'

In [31]:
missing_counties=missing_counties.drop(columns=['sab', 'cname', 'fips'])

In [32]:
missing_counties=missing_counties.merge(county_fips, left_on=['county','st'], right_on=['cname','sname'], how='left')

In [33]:
mc=missing_counties[missing_counties['fips'].isnull()]

In [47]:
mc_shortfips=missing_counties[missing_counties['fips'].str.len()<5].copy()

In [48]:
mc_longfips=missing_counties[missing_counties['fips'].str.len()>4].copy()

In [49]:
mc_shortfips['fips']='0'+mc_shortfips['fips']

In [37]:
mc_tomerge=mc_longfips[['county','pediatricians', 'psychiatrists', 'fm_physicians',
       'licensed_sw', 'psychologists', 'st','fips']]

In [38]:
final_providers_counties=providers_counties_fips[['county','pediatricians', 'psychiatrists', 'fm_physicians',
       'licensed_sw', 'psychologists', 'st','fips']]

In [39]:
final_providers_counties=pd.concat([mc_tomerge, final_providers_counties])

#### Add AP states

In [40]:
name_to_ap = us.states.mapping('name', 'ap_abbr')

In [41]:
final_providers_counties['ap_st'] = final_providers_counties['st'].map(name_to_ap)

In [42]:
final_providers_counties.head()

,county,pediatricians,psychiatrists,fm_physicians,licensed_sw,psychologists,st,fips,ap_st
7,Miami-Dade,11.4,5.3,10.0,24.9,14.4,Florida,12086,Fla.
8,St. Johns,8.8,2.7,18.8,15.0,7.3,Florida,12109,Fla.
9,St. Lucie,5.0,3.0,7.4,29.3,4.3,Florida,12111,Fla.
10,St. Clair,5.0,1.4,15.6,11.5,2.5,Illinois,17163,Ill.
11,St. Bernard,0.8,0.0,3.4,72.1,2.5,Louisiana,22087,La.


In [43]:
dw.add_data('7MF9g', final_providers_counties)

<Response [204]>

In [44]:
#dfs_list=[]

In [45]:
#for sn in states:
# dfs_list.append(pd.read_html(f"https://www.cdc.gov/childrensmentalhealth/stateprofiles-providers/{sn}/index.html")[0])

In [46]:
#df=pd.concat([dfs_list]).assign(state=sn)